In [1]:
import scanpy as sc

In [2]:
import cpa
import scanpy as sc

[rank: 0] Global seed set to 0
2024-04-15 23:14:21,545	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-04-15 23:14:29,950	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [3]:
from cpa import CPA

In [4]:
import gdown
#gdown.download('https://drive.google.com/uc?export=download&id=1RRV0_qYKGTvD3oCklKfoZQFYqKJy4l6t')
data_path = 'combo_sciplex_prep_hvg_filtered.h5ad'
adata = sc.read(data_path)

In [5]:
adata_train_path = "/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_train.h5ad"
adata_test_path = "/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_test.h5ad"
adata_ood_path = "/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_ood.h5ad"

In [6]:
adata_train = sc.read(adata_train_path)
adata_test = sc.read(adata_test_path)
adata_ood = sc.read(adata_ood_path)


In [7]:
adata_train.X = adata_train.layers["counts"]
adata_test.X = adata_test.layers["counts"]
adata_test.X = adata_test.layers["counts"]

In [8]:
adata_train.obs["cov_drug_dose"] = adata.obs['cov_drug_dose']
adata_train.obs["condition_ID"] = adata.obs["condition_ID"]
adata_train.obs["log_dose"] = adata.obs["log_dose"]

adata_test.obs["cov_drug_dose"] = adata.obs['cov_drug_dose']
adata_test.obs["condition_ID"] = adata.obs["condition_ID"]
adata_test.obs["log_dose"] = adata.obs["log_dose"]

adata_ood.obs["cov_drug_dose"] = adata.obs['cov_drug_dose']
adata_ood.obs["condition_ID"] = adata.obs["condition_ID"]
adata_ood.obs["log_dose"] = adata.obs["log_dose"]

In [9]:

adata.obs["index"] = adata.obs_names.values
adata.obs["index_adapted"] = adata.obs.apply(lambda x: x["index"][:-2], axis=1)
adata.obs.set_index("index_adapted", inplace=True)

In [10]:
adata.obs.set_index("index_adapted", inplace=True)

In [11]:
frac_valid = adata[adata.obs['split_1ct_MEC']=="valid"].n_obs/(adata[adata.obs['split_1ct_MEC']=="valid"].n_obs+adata[adata.obs['split_1ct_MEC']=="train"].n_obs)

In [12]:
import numpy as np

def create_split(x):
    is_train = np.random.choice(2, p=[frac_valid, 1-frac_valid])
    if is_train:
        return "train"
    return "valid"
adata_train.obs['split_1ct_MEC'] = adata_train.obs.apply(create_split, axis=1)

In [13]:
adata_train.obs['split_1ct_MEC'].value_counts()

split_1ct_MEC
train    46259
valid     5242
Name: count, dtype: int64

In [20]:
adata_train.obs['split_1ct_MEC'] = adata_train.obs['split_1ct_MEC'].astype("category")

In [14]:
adata_train.obs['condition_ID'] = adata_train.obs['condition_ID'].cat.remove_unused_categories()

In [21]:
cpa.CPA.load('/home/icb/dominik.klein/tmp', adata_train)

INFO     File /home/icb/dominik.klein/tmp/model.pt already downloaded                                              


TypeError: sequence item 1: expected str instance, float found

In [ ]:
model.predict(adata_train)

In [17]:
cpa.CPA.setup_anndata(adata_train,
                      perturbation_key='condition_ID',
                      dosage_key='log_dose',
                      control_group='CHEMBL504',
                      batch_key=None,
                      is_count_data=True,
                      categorical_covariate_keys=['cell_type'],
                      #deg_uns_key='rank_genes_groups_cov',
                      deg_uns_cat_key='cov_drug_dose',
                      max_comb_len=2,
                     )

TypeError: sequence item 1: expected str instance, float found

In [22]:
def compute_r_squared(x: np.ndarray, y: np.ndarray) -> float:
    return r2_score(np.mean(x, axis=0), np.mean(y, axis=0))

In [26]:

preds_train_log = adata_train.X.A - adata_train.varm["X_train_mean"].T
preds_train_pca = np.matmul(preds_train_log, adata_train.varm["PCs"])

In [28]:
from sklearn.metrics import r2_score
compute_r_squared(preds_train_pca, adata_train.obsm["X_pca"])

-222.93512590759994

In [174]:
ae_hparams = {
    "n_latent": 128,
    "recon_loss": "nb",
    "doser_type": "logsigm",
    "n_hidden_encoder": 512,
    "n_layers_encoder": 3,
    "n_hidden_decoder": 512,
    "n_layers_decoder": 3,
    "use_batch_norm_encoder": True,
    "use_layer_norm_encoder": False,
    "use_batch_norm_decoder": True,
    "use_layer_norm_decoder": False,
    "dropout_rate_encoder": 0.1,
    "dropout_rate_decoder": 0.1,
    "variational": False,
    "seed": 434,
}

trainer_params = {
    "n_epochs_kl_warmup": None,
    "n_epochs_pretrain_ae": 30,
    "n_epochs_adv_warmup": 50,
    "n_epochs_mixup_warmup": 3,
    "mixup_alpha": 0.1,
    "adv_steps": 2,
    "n_hidden_adv": 64,
    "n_layers_adv": 2,
    "use_batch_norm_adv": True,
    "use_layer_norm_adv": False,
    "dropout_rate_adv": 0.3,
    "reg_adv": 20.0,
    "pen_adv": 20.0,
    "lr": 0.0003,
    "wd": 4e-07,
    "adv_lr": 0.0003,
    "adv_wd": 4e-07,
    "adv_loss": "cce",
    "doser_lr": 0.0003,
    "doser_wd": 4e-07,
    "do_clip_grad": False,
    "gradient_clip_value": 1.0,
    "step_size_lr": 45,
}

In [175]:
model = cpa.CPA(adata=adata_train,
                split_key='split_1ct_MEC',
                train_split='train',
                valid_split='valid',
                **ae_hparams,
               )

[rank: 0] Global seed set to 434


In [176]:
model.train(max_epochs=2000,
            use_gpu=True,
            batch_size=128,
            plan_kwargs=trainer_params,
            early_stopping_patience=10,
            check_val_every_n_epoch=5,
            save_path='/home/icb/dominik.klein/tmp',
           )


100%|██████████| 27/27 [00:00<00:00, 180.58it/s]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA A100-PCIE-40GB MIG 3g.20gb') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [MIG-d2c9e8c7-5bf5-5039-b1c7-e062977cb38b]


Epoch 5/2000:   0%|          | 4/2000 [01:39<13:45:47, 24.82s/it, v_num=1, recon=324, r2_mean=0.757, adv_loss=2.06, acc_pert=0.264] 


Epoch 00004: cpa_metric reached. Module best state updated.


Epoch 10/2000:   0%|          | 9/2000 [03:48<13:53:36, 25.12s/it, v_num=1, recon=305, r2_mean=0.77, adv_loss=1.87, acc_pert=0.312, val_recon=316, disnt_basal=0.0677, disnt_after=0.0968, val_r2_mean=0.76, val_KL=nan]


Epoch 00009: cpa_metric reached. Module best state updated.


...

Epoch 20/2000:   1%|          | 19/2000 [08:04<13:45:23, 25.00s/it, v_num=1, recon=288, r2_mean=0.777, adv_loss=1.82, acc_pert=0.336, val_recon=294, disnt_basal=0.0569, disnt_after=0.0823, val_r2_mean=0.767, val_KL=nan]


Epoch 00019: cpa_metric reached. Module best state updated.



disnt_basal = 0.05217186558849538
disnt_after = 0.07826841322439168
val_r2_mean = 0.7692084657295386
val_r2_var = 0.1581302106541998
Epoch 30/2000:   1%|▏         | 29/2000 [12:20<13:43:50, 25.08s/it, v_num=1, recon=279, r2_mean=0.78, adv_loss=1.82, acc_pert=0.34, val_recon=285, disnt_basal=0.0498, disnt_after=0.0763, val_r2_mean=0.764, val_KL=nan]  
disnt_basal = 0.04738376711187475
disnt_after = 0.07347231022064356
val_r2_mean = 0.7687046536385546
val_r2_var = 0.1558590970591475
Epoch 40/2000:   2%|▏         | 39/2000 [16:25<13:01:58, 23.93s/it, v_num=1, recon=277, r2_mean=0.78, adv_loss=3.4, acc_pert=0.0563, val_recon=283, disnt_basal=0.0373, disnt_after=0.0631, val_r2_mean=0.767, val_KL=nan]  
disnt_basal = 0.03737263614007471
disnt_after = 0.06495484860067124
val_r2_mean = 0.770815257054498
val_r2_var = 0.14688270957315835
Epoch 45/2000:   2%|▏         | 44/2000 [18:25<12:53:33, 23.73s/it, v_num=1, recon=276, r2_mean=0.781, adv_loss=3.34, acc_pert=0.0457, val_recon=282, disnt_bas


Epoch 00044: cpa_metric reached. Module best state updated.


Epoch 50/2000:   2%|▏         | 49/2000 [20:26<12:52:57, 23.77s/it, v_num=1, recon=275, r2_mean=0.78, adv_loss=3.32, acc_pert=0.0454, val_recon=281, disnt_basal=0.0363, disnt_after=0.065, val_r2_mean=0.772, val_KL=nan] 
disnt_basal = 0.035491545077025226
disnt_after = 0.06455249115751736
val_r2_mean = 0.7676875444353733
val_r2_var = 0.14250919413451318
Epoch 60/2000:   3%|▎         | 59/2000 [24:30<12:49:24, 23.78s/it, v_num=1, recon=274, r2_mean=0.782, adv_loss=3.3, acc_pert=0.0468, val_recon=280, disnt_basal=0.0354, disnt_after=0.064, val_r2_mean=0.767, val_KL=nan]  
disnt_basal = 0.03522260966887392
disnt_after = 0.06311117509432769
val_r2_mean = 0.7692925417376653
val_r2_var = 0.14895800763207281
Epoch 70/2000:   3%|▎         | 69/2000 [28:32<12:46:24, 23.81s/it, v_num=1, recon=273, r2_mean=0.783, adv_loss=3.29, acc_pert=0.0467, val_recon=280, disnt_basal=0.0333, disnt_after=0.0625, val_r2_mean=0.769, val_KL=nan]
disnt_basal = 0.034739183793282793
disnt_after = 0.0635861702116866
v

In [177]:
cpa.CPA.setup_anndata(adata_test,
                      perturbation_key='condition_ID',
                      dosage_key='log_dose',
                      control_group='CHEMBL504',
                      batch_key=None,
                      is_count_data=True,
                      categorical_covariate_keys=['cell_type'],
                      #deg_uns_key='rank_genes_groups_cov',
                      deg_uns_cat_key='cov_drug_dose',
                      max_comb_len=2,
                     )

100%|██████████| 3100/3100 [00:00<00:00, 417853.34it/s]


INFO     Generating sequential column names                                                                        
INFO     Generating sequential column names                                                                        


In [178]:
cpa.CPA.setup_anndata(adata_ood,
                      perturbation_key='condition_ID',
                      dosage_key='log_dose',
                      control_group='CHEMBL504',
                      batch_key=None,
                      is_count_data=True,
                      categorical_covariate_keys=['cell_type'],
                      #deg_uns_key='rank_genes_groups_cov',
                      deg_uns_cat_key='cov_drug_dose',
                      max_comb_len=2,
                     )

100%|██████████| 9277/9277 [00:00<00:00, 917766.78it/s]

INFO     Generating sequential column names                                                                        
INFO     Generating sequential column names                                                                        


In [179]:
model.predict(adata_test)

INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             


100%|██████████| 97/97 [00:00<00:00, 127.72it/s]


In [180]:
preds = adata_test.obsm["CPA_pred"]

In [183]:
preds_pca = np.matmul(preds, adata_train.varm["PCs"])

In [192]:
model.predict(adata_train)

100%|██████████| 1610/1610 [00:30<00:00, 51.96it/s]


In [193]:
preds_train = adata_train.obsm["CPA_pred"]

In [199]:
adata_train.X.shape, preds_train.shape

((51501, 2000), (51501, 2000))

In [200]:
compute_r_squared(preds_train, adata_train.X.A)

0.9963734489445446

In [215]:
preds_train_log = np.log1p(preds_train) - adata_train.varm["X_train_mean"].T
preds_train_pca = np.matmul(preds_train_log, adata_train.varm["PCs"])

In [223]:
(np.log1p(preds_train).mean(axis=0) - adata_train.varm["X_train_mean"]).mean()

-0.015940763

In [221]:
adata_train.varm["X_train_mean"]

array([[0.10696691],
       [0.01764269],
       [0.07884356],
       ...,
       [0.00024302],
       [0.00037494],
       [0.00057457]], dtype=float32)

In [216]:
preds_train_pca = np.matmul(preds_train_log, adata_train.varm["PCs"])

In [ ]:
true_train_pca = np.matmul(adata_train, adata_train.varm["PCs"])

In [218]:
compute_r_squared(preds_train_pca, adata_train.obsm["X_pca"])

-0.01202978864684412

In [219]:
preds_train[0].min(), preds_train[0].max()

(7.5745454e-10, 5.176859)

In [186]:
model.predict(adata_ood)

INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             


ValueError: Category CHEMBL1213492+CHEMBL257991 not found in source registry. Cannot transfer setup without `extend_categories = True`.

In [ ]:
preds_ood = adata_ood.obsm["CPA_pred"]

In [190]:
"CHEMBL257991" in adata_train.obs["condition_ID"]

False

In [ ]:
"CHEMBL1213492" in adata_train.obs[]

In [138]:
preds_pca = np.matmul(preds, adata_train.varm["PCs"])

In [140]:
def compute_r_squared(x: np.ndarray, y: np.ndarray) -> float:
    return r2_score(np.mean(x, axis=0), np.mean(y, axis=0))

In [153]:
np.isfinite(preds).shape

(3100, 2000)

In [142]:
from sklearn.metrics import r2_score
compute_r_squared(preds_pca, adata_test.obsm["X_pca"])

Epoch 3/2000:   0%|          | 2/2000 [33:26<556:44:02, 1003.12s/it, v_num=1, recon=1.58e+3, r2_mean=0.453, adv_loss=2.55, acc_pert=0.173]


ValueError: Input contains NaN.

In [149]:
(~np.isnan(preds)).sum()

1492000

In [144]:
preds_pca.shape


(3100, 30)

In [145]:
adata_train.varm["PCs"].shape

(2000, 30)

In [156]:
mask = ~np.isnan(preds).any(axis=1)

In [158]:
arr_without_nan = preds[mask]

In [160]:
arr_without_nan.shape

(746, 2000)

In [161]:
preds_pca = np.matmul(arr_without_nan, adata_train.varm["PCs"])

In [184]:
compute_r_squared(preds_pca, adata_test.obsm["X_pca"])

-0.028193542538006877